In [1]:
import math                                 #for math.ceil()
from typing import List, Dict               #for annotation

#Function, that return file content as list of string
def Read_file(fname : str)->List[str]:
    f = None
    data = []
    try:
        f = open(fname)
        for line in f:
            data.append(line)
    finally:
        if f != None:
            f.close()
        else:
            print('File error')
        return data
    
#The Read_block() function return List of string contain sub-block starting at prescribed line, 
#define type of sub-block and call necessary reading function

def Read_block(NLine:int, data:List[str])->List[str]:
    #get block id from first two numbers in second line
    NE  = int(data[NLine+1][0:3])
    NTE = int(data[NLine+1][3:6])
    print(NE,NTE)
    if NE == 0 and NTE == 0:
        print('Airfoil')
        ret_data = Read_Airfoil(NLine, data)
    if NE != 0:
        if abs(NTE) == 1:
            print('Body at line: ', NLine)
            ret_data = Read_Body(NLine, data)
        if abs(NTE) == 2:
            print('Wing at line: ', NLine)
            ret_data = Read_Wing(NLine, data)
        if abs(NTE) == 3:
            print('Engine nacelle at line: ', NLine)
            ret_data = Read_Nacelle(NLine, data)
        if abs(NTE) == 4: 
            print('Jet at line:', NLine)
            ret_data = Read_Jet(NLine, data)
    if NE < 0:
        print('This is a last item in the file at line: ', NLine)
    return ret_data

#The Control_Line() function return a list of int with control numbers from string passed in
def Control_Line(cstr:str)->List[int]:
    cList = [0]*20             #20 seems more than enough, manual is report at max 17 numbers in control string
    cstr = cstr.rstrip()       #trim right side spaces
    maxlen = math.ceil(len(cstr)/3)
    for i in range(maxlen):
        try:
            cList[i] = int(cstr[0:3])
        except:
            pass
        cstr = cstr[3:]
    return cList

#Τhe Read_Airfoil() function calculate lines count of airfoil block and return these lines
def Read_Airfoil(NLine:int, data:List[str])->List[str]:
    #Read second line in block, sequence of int
    cList = Control_Line(data[NLine+1])
    NP = cList[2]      #Number of airfoil
    ITBE = cList[3]    #Specify type of coordinate defenition
    IK = cList[4]      #Leading edge index ???
    NU = cList[5]      #Point count
    IS = cList[6]      #Symmetry flag
    IM = cList[7]      #Rescale flag
    IP = cList[8]      #Rotation flag
    #Block size calculation
    LineCount = 3      #Header, control line, commentary
    if ITBE in (0,1):
        n = 3
    if ITBE == 2:
        n = 2
    LineCount += n+n*math.ceil(NU/10)
    if IM == 1 or IP == 1:
        LineCount +=2
    return data[NLine:(NLine+LineCount)]

#Τhe Read_Body() function calculate lines count of body (fuselage, some nacelle) block and return these lines
def Read_Body(NLine:int, data:List[str])->List[str]:
    #Read second line in block, sequence of int
    cList = Control_Line(data[NLine+1])
    ITBE   = cList[2]    #Specify type of coordinate definition
    NSE    = cList[3]    #Sections count
    NU     = cList[4]    #Points per section
    IIPPC  = cList[5]    #Change sections flag
    NSEI   = cList[6]    #New sections count
    INUT   = cList[7]    #Change points flag
    NUI    = cList[8]    #New points count
    ICHAR  = cList[9]    #Reference values for normalization
    IMASH  = cList[10]   #Rescale flag
    IPREOB = cList[11]   #Rotate and translate
    #Block size calculation
    LineCount = 3      #Header, control line, commentary
    if ITBE == 0:
        LineCount += 1
        LineCount += 3*NSE*math.ceil(NU/10)
    elif ITBE == 1:
        LineCount += 1
        LineCount += math.ceil(NSE/10)
        LineCount += 1
        LineCount += 2*NSE*math.ceil(NU/10)
    elif ITBE == 2:
        LineCount += 4
        LineCount += 4*math.ceil(NSE/10)
    if IIPC != 0 and NSEI != 0:
        LineCount += 1
        LineCount += math.ceil(NSEI/10)
    #If Reference values is specify
    if ICHAR != 0:
        LineCount += 2
    #Rescale
    if IMASH != 0:
        LineCount += 2
    #Rotate and translate
    if IPREOB != 0:
        LineCount += 2
    return data[NLine:(NLine+LineCount)]

#The Read_Wing() function calculate lines count of wing block and return these lines
def Read_Wing(NLine:int, data:List[str])->List[str]:
    #Read second line in block, sequence of int
    cList = Control_Line(data[NLine+1])
    ITBE   = cList[2]     #Wing definition type
    NSE    = cList[3]     #Spanwise section count
    NU     = cList[4]     #Points chordwise
    IIPC   = cList[5]     #Reinterpolate section
    NSEI   = cList[6]     #New section count
    INUI   = cList[7]     #don't used for wing
    NLIP   = cList[8]     #wake panel count per strip
    ICHAR  = cList[9]     #Reference values for normalization
    IMASH  = cList[10]    #Rescale flag
    IPREOB = cList[11]    #Resize and translate
    #Block size calculation
    LineCount = 3      #Header, control line, commentary
    LineCount += NSE*2
    if NU > 0:
        LineCount += 1+math.ceil(NU/10)
    if IIPC == 1 and NSEI != 0:
        LineCount += 1+math.ceil(NSEI/10)
    #If Reference values is specify
    if ICHAR != 0:
        LineCount += 2
    #Rescale
    if IMASH != 0:
        LineCount += 2
    #Rotate and translate
    if IPREOB != 0:
        LineCount += 2
    return data[NLine:(NLine+LineCount)]    

#The Read_Nacelle() function calculate lines count of nacelle block and return these lines
def Read_Nacelle(NLine:int, data:List[str])->List[str]:
    #Read second line in block, sequence of int
    cList = Control_Line(data[NLine+1])
    ITBE   = cList[2]     #Nacelle definition type
    NSE    = cList[3]     #Nacelle section count
    NU     = cList[4]     #Points per section
    IIPC   = cList[5]     #Reinterpolate section
    NSEI   = cList[6]     #New section count
    NUI   = cList[7]      #New point per section
    NLIP   = cList[8]     #wake panel count per strip
    ICHAR  = cList[9]     #Reference values for normalization
    IMASH  = cList[10]    #Rescale flag
    IPREOB = cList[11]    #Resize and translate
    #Block size calculation
    LineCount = 3      #Header, control line, commentary
    if ITBE == 1:
        LineCount += 3 + 3*math.ceil(NSE/10)
        LineCount += (1+ 4*math.ceil(NU/10))*NSE
    #If Reference values is specify
    if ICHAR != 0:
        LineCount += 2
    #Rescale
    if IMASH != 0:
        LineCount += 2
    #Rotate and translate
    if IPREOB != 0:
        LineCount += 2
    return data[NLine:(NLine+LineCount)]    

#The Read_Jet() function calculate lines count of jet block and return these lines
def Read_Jet(NLine:int, data:List[str])->List[str]:
    #Read second line in block, sequence of int
    cList = Control_Line(data[NLine+1])
    ITBE   = cList[2]     #Jet definition type
    NSE    = cList[3]     #Jet section count
    NU     = cList[4]     #Points per section
    IIA    = cList[5]     #Active section flag
    NSEA   = cList[6]     #Nacelle section N with jet active section
    NRRA   = cList[7]     #Points per active section radius
    ISJ    = cList[8]     #Jet symmetry flag
    NJOHV  = cList[9]     #Outer jet number
    IEJ    = cList[10]    #Jet coupled with nacelle flag
    NEG    = cList[11]    #Nacelle N coupled with jet
    #Block size calculation
    LineCount = 3      #Header, control line, commentary
    if ITBE == 1:
        LineCount += 2
    if IIA == 1:
        LineCount += 2
    return data[NLine:(NLine+LineCount)]    

In [2]:
#Startline position
N_START = 6-1
#Blocknumber
N = 1
#Empty dictionary of blocks
aDict = {int:List[str]}
#Filename
folder = "D:\\Проекты\\Аэропракт А-8 в PANSYM\\Расчетные модели PANSYM"
filename = folder + "\\A8.in"
#Read 1st block and add to dictionary
data_item = Read_block(N_START, Read_file(filename))
aDict[N] = data_item
#Read next if exist
while (len(data_item)!=0) and (int(data_item[1][0:3])>=0):
    N_START += len(data_item)
    data_item = Read_block(N_START, Read_file(filename))
    N+=1
    if data_item == []:
        break
    aDict[N]=data_item
    if int(data_item[1][0:3])<0:
        break

print()
print('Print header of all readed blocks: ')        
print()
for i in range(1,len(list(aDict.keys()))):
    print(aDict[i][0])
print()
print('Last line of last blocks:')
print(aDict[len(list(aDict.keys()))-1][-1])
    

0 0
Airfoil
0 0
Airfoil
1 2
Wing at line:  38
-2 2
Wing at line:  48
This is a last item in the file at line:  48

Print header of all readed blocks: 

RAF-32

FX-60-126

#First_wing

#Second_wing


Last line of last blocks:
  .6    .7     .8    .9      .95    1.0

